In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()

%time mnist = input_data.read_data_sets("data/", one_hot=True)  # %time을 통해 전체 실행 시간을 남길 수 있습니다.
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W0722 09:05:36.158478 15692 deprecation.py:323] From <timed exec>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0722 09:05:36.161470 15692 deprecation.py:323] From D:\anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0722 09:05:36.163465 15692 deprecation.py:323] From D:\anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality

Extracting data/train-images-idx3-ubyte.gz


W0722 09:05:36.493609 15692 deprecation.py:323] From D:\anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0722 09:05:36.501562 15692 deprecation.py:323] From D:\anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0722 09:05:36.571376 15692 deprecation.py:323] From D:\anaconda\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please 

Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Wall time: 728 ms


In [1]:
def reset_all(seed=0):
    # 왜 할까요? 생각해봅시다.
    tf.reset_default_graph() # graph 초기화
    tf.set_random_seed(seed) # tf.random_seed 초기화
    np.random.seed(seed) # np.random_seed 초기화

In [2]:
reset_all()

NameError: name 'tf' is not defined

In [2]:
""" placeholder 정의 : 데이터가 들어 갈 곳
이미지와 정답 레이블용 2차원 tensor를 만든다.
None은 어떤 length도 가능함을 의미한다. """
# 이미지 데이터용 placeholder
with tf.name_scope("input") as scope:
    x = tf.placeholder(tf.float32, [None, 784])
    #print(x.shape) #(?, 784)
# 정답 레이블용 placeholder
with tf.name_scope("y_") as scope:
    y_ = tf.placeholder(tf.float32, [None, 10])
    #print(y_.shape) #(?, 10)

""" Variable 정의 : 학습 결과가 저장될 가중치(weight)와 바이어스(bias) """
# 0으로 초기화 함
with tf.name_scope("weight") as scope:
    W = tf.Variable(tf.zeros([784, 10]))
    #print(W.shape) #(784, 10)
with tf.name_scope("bias") as scope:
    b = tf.Variable(tf.zeros([10]))
    #print(b.shape) #(10,)

""" 모델 정의 : Softmax Regression
10개의 값 중 가장 확률이 높은 것을 고르기 위해 Softmax 사용 """
# 모델 생성
with tf.name_scope("layer1") as scope:
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    #print(y.shape) #(?, 10)

w_hist = tf.summary.histogram("weight", W)
b_hist = tf.summary.histogram("bias", b)
y_hist = tf.summary.histogram("y", y)

In [ ]:
""" 모델 훈련 """
# Loss 함수 정의
with tf.name_scope("cost") as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    #tf.summary.scalar("cost",cross_entropy)
    cost_sum = tf.summary.scalar("cost",cross_entropy)
# learning rate을 0.5로 정의
with tf.name_scope("train") as scope:
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Merge all summaries into a single op
merged = tf.summary.merge_all()

# 세션 시작전에 모든 변수 초기화함
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

# 기본으로 제공되는 텐서보드 로그 dir : /home/ncp/workspace/tensorboard
writer =tf.summary.FileWriter("./tensorboard", sess.graph)

# Training : 100개 단위로 샘플링하여 1000회 학습 진행
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)  # 학습 데이터셋에서 무작위로 샘플링한 100개의 데이터로 구성된 'batch'를 가져옴
    #print(batch_xs.shape) #(100, 784)
    #print(batch_ys.shape) #(100, 10)
    summary, _ = sess.run([merged, train_step], feed_dict={x: batch_xs, y_: batch_ys})  # placeholder x, y_에 샘플링된 batch_xs, batch_ys를 공급함
    # write summary events to disk
    writer.add_summary(summary,i)

In [ ]:
""" 모델 평가 """
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 정확도
print (sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# 실행을 모두 마치면 Session을 닫음
writer.close()
sess.close()